# Basic cprofile & memory profile 

* Profile is a log of transactions that occur while a program is compiled 
* Profilers are one of the tools used to analyse the performance of a program
* It can help us answer questions such as:
 * how much memory is the program using?
 * how much time is spent on functions?
 * how much time is spent on statements?

* **Common Applications**: Optimization/Cleanups, Architecture Design etc.

* This notebook presents 
 * Types of profiling
 * How to use them?
 * Example

## Types of Profiling

**Run-Time Profiling**
* Tracks Function calls or Track statements 
    * function : n_calls
    * function : Run time per call etc.
* Function profiling
* Line Profiling
* Related with Time complexity

*Example module* : cprofile, line_profile

**Memory Profiling** 
* Tracks memory wrt time or execution flow
* Line vs Total Memory Occupied
* Related with space complexity

*Example module* : memory_profiler


## Using Profilers

Python profilers are inbuilt plug and play modules

1. Import them
2. Embed the relevant properties into the target piece of code 
3. Execute the code as usual 
4. Report is generated at the end of execution 

Use this report to find out the major consumers/bottlenecks of runtime or memory, Try to derive possible connections in the bottleneck pieces

In [3]:
#Example Problem: 

"""
Given a dataset of 5000 movie titles, the program needs to count the duplicates in them

Flow: 
We'll start with a bad code, analyse it and improve it
"""

# Using cProfiler to optimize runtime
# Use memory profiler to analyze the memory
import time

In [5]:
# Initial Program : Version 0
start = time.time()

def read_movies(src):
    with open(src) as datafile:
        return datafile.read().splitlines()

def is_duplicate(needle,haystack):
    for movie in haystack:
        if needle.lower() == movie.lower():
            return True
    return False

def find_duplicate_movies(src= 'movie_metadata.txt'):
    movies = read_movies(src)
    duplicates = []
    while movies:
        this_movie = movies.pop()
        if is_duplicate(this_movie,movies):
            duplicates.append(this_movie)
    return duplicates

duplicates = find_duplicate_movies()
print ("Number of Duplicates : ",len(duplicates))

end = time.time()
print("exec_time = ",end-start," sec." )

Number of Duplicates :  127
exec_time =  2.6814932823181152  sec.


--------------------------------------------------------------------------------------------------------------------------
We know by experience that MS-excel, or pandas drop duplicates function does the job of searching duplicates in the fraction of a second, Plus we are using a high performance machine to run this code, There's difinitely something wrong with this program that it takes 2 sec. 

Lets analyse the code with cprofile and find the major chunk of time consumption

To apply the profile we import cprofile as a python module and embed the program between the ```profile.start()``` and ```profile.end()``` methods as illustrated below. This can be done for complete code or just a piece between the full program

*cprofile module generates the profile & pstats, io format the log and handle read/write to a file* 

**Note:** crpofile has an overhead, it increases the execution time

In [7]:
# Cprofile
import cProfile, pstats                    #
from io import StringIO                    # COMMENT/UNCOMMENT THESE LINES
pr = cProfile.Profile()                    # TO ENGAGE/START PROFILE
pr.enable()                                #

############################## YOUR CODE STARTS HERE ################################################################

# Program : Version 0 
start = time.time()

def read_movies(src):
    with open(src) as datafile:
        return datafile.read().splitlines()

def is_duplicate(needle,haystack):
    for movie in haystack:
        if needle.lower() == movie.lower():
            return True
    return False

def find_duplicate_movies(src= 'movie_metadata.txt'):
    movies = read_movies(src)
    duplicates = []
    while movies:
        this_movie = movies.pop()
        if is_duplicate(this_movie,movies):
            duplicates.append(this_movie)
    return duplicates

duplicates = find_duplicate_movies()
print ("Number of Duplicates : ",len(duplicates))

end = time.time()
print("exec_time = ",end-start," sec." )

############################# YOUR CODE ENDS HERE ###################################################################

pr.disable()                                            #
s = StringIO()                                          #
sortby = 'cumulative'                                   #
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)      # COMMENT/UNCOMMENT THESE
ps.print_stats()                                        # TO DISENGAGE/END PROFILE
print(s.getvalue())                                     #

Number of Duplicates :  127
exec_time =  5.065772771835327  sec.
         24980616 function calls in 5.067 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.000    0.000    5.067    0.563 c:\users\admin\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:3230(run_code)
        9    0.000    0.000    5.067    0.563 {built-in method builtins.exec}
        1    0.000    0.000    5.066    5.066 <ipython-input-7-9c3afe065c91>:31(<module>)
        1    0.003    0.003    5.066    5.066 <ipython-input-7-9c3afe065c91>:22(find_duplicate_movies)
     5043    3.014    0.001    5.061    0.001 <ipython-input-7-9c3afe065c91>:16(is_duplicate)
 24970168    2.047    0.000    2.047    0.000 {method 'lower' of 'str' objects}
        1    0.000    0.000    0.002    0.002 <ipython-input-7-9c3afe065c91>:12(read_movies)
     5043    0.001    0.000    0.001    0.000 {method 'pop' of 'list' obj

## How to read the report

* filename/lineno : The function or line against which the transaction data is listed 
* ncalls : Total number of times the corresponding function is called  
* percall : Time consumed per call (in sec.)
* cumtime : Cumulative Time consumed during the execution (in sec) 


## Observation

* First two methods are related with the execution of program 
* ```find_duplicate_movies()``` is the main function which is called 
* Further ```is_duplicate()``` which lies inside ```find_duplicate_movies()``` is the top runtime consumer (makes sense) 
* Inside ```is_duplicate()``` we have ```str.lower()``` which is evident in crpofile as well in code and then there's a sudden drop of percall time 
* Based on the connections drawn above we figure out that ```lower()``` is taking maximum time during the execution 

## Conclusion 

Think further, we are calling lower function multiple times for the same item in the list which is not required we can directly lower case all at once before comparing and wouldnt need to call that again and again

Following is the improved version of program

In [9]:
# Cprofile
import cProfile, pstats                    #
from io import StringIO                    # COMMENT/UNCOMMENT THESE LINES
pr = cProfile.Profile()                    # TO ENGAGE/START PROFILE
pr.enable()                                #

############################## YOUR CODE STARTS HERE ################################################################

# Program : Version 1 
start = time.time()

def read_movies(src):
    with open(src) as datafile:
        return datafile.read().splitlines()

def is_duplicate(needle,haystack):
    for movie in haystack:
        if needle == movie:     # Changed Here
            return True
    return False

def find_duplicate_movies(src= 'movie_metadata.txt'):
    movies = read_movies(src)
    movies = [movie.lower() for movie in movies]  # Changed Here 
    duplicates = []
    while movies:
        this_movie = movies.pop()
        if is_duplicate(this_movie,movies):
            duplicates.append(this_movie)
    return duplicates

duplicates = find_duplicate_movies()
print ("Number of Duplicates : ",len(duplicates))

end = time.time()
print("exec_time = ",end-start," sec." )

############################# YOUR CODE ENDS HERE ###################################################################

pr.disable()                                            #
s = StringIO()                                          #
sortby = 'cumulative'                                   #
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)      # COMMENT/UNCOMMENT THESE
ps.print_stats()                                        # TO DISENGAGE/END PROFILE
print(s.getvalue())                                     #

Number of Duplicates :  127
exec_time =  0.34298062324523926  sec.
         15492 function calls in 0.347 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.000    0.000    0.347    0.039 c:\users\admin\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:3230(run_code)
        9    0.000    0.000    0.347    0.039 {built-in method builtins.exec}
        1    0.000    0.000    0.347    0.347 <ipython-input-9-36e88931b00d>:32(<module>)
        1    0.002    0.002    0.347    0.347 <ipython-input-9-36e88931b00d>:22(find_duplicate_movies)
     5043    0.341    0.000    0.341    0.000 <ipython-input-9-36e88931b00d>:16(is_duplicate)
        1    0.000    0.000    0.002    0.002 <ipython-input-9-36e88931b00d>:12(read_movies)
        1    0.001    0.001    0.001    0.001 <ipython-input-9-36e88931b00d>:24(<listcomp>)
        1    0.001    0.001    0.001    0.001 {method 'read' o

## Observation 

*Eureka!* We have reduced execution time from 5 sec to 0.34 sec just with a small change. That's how cprofile helps to identify the pieces where you need the improvement on prioirty 

Futher observation is that there's still a ```is_duplicate()``` which consumes a major chunk, Let's try eliminating ```is_duplicate()```

In [10]:
# Cprofile
import cProfile, pstats                    #
from io import StringIO                    # COMMENT/UNCOMMENT THESE LINES
pr = cProfile.Profile()                    # TO ENGAGE/START PROFILE
pr.enable()                                #

############################## YOUR CODE STARTS HERE ################################################################

# Program : Version 2
start = time.time()

def read_movies(src):
    with open(src) as datafile:
        return datafile.read().splitlines()

# def is_duplicate(needle,haystack):    # Changed here 
#     for movie in haystack:
#         if needle == movie:     
#             return True
#     return False

def find_duplicate_movies(src= 'movie_metadata.txt'):
    movies = read_movies(src)
    movies = [movie.lower() for movie in movies]  
    duplicates = []
    while movies:
        this_movie = movies.pop()
        if this_movie in movies:       # Changed Here 
            duplicates.append(this_movie)
    return duplicates

duplicates = find_duplicate_movies()
print ("Number of Duplicates : ",len(duplicates))

end = time.time()
print("exec_time = ",end-start," sec." )

############################# YOUR CODE ENDS HERE ###################################################################

pr.disable()                                            #
s = StringIO()                                          #
sortby = 'cumulative'                                   #
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)      # COMMENT/UNCOMMENT THESE
ps.print_stats()                                        # TO DISENGAGE/END PROFILE
print(s.getvalue())             

Number of Duplicates :  127
exec_time =  0.1500077247619629  sec.
         10440 function calls in 0.159 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.000    0.000    0.158    0.020 c:\users\admin\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:3230(run_code)
        8    0.000    0.000    0.158    0.020 {built-in method builtins.exec}
        1    0.000    0.000    0.158    0.158 <ipython-input-10-d49f76dd3d33>:32(<module>)
        1    0.154    0.154    0.158    0.158 <ipython-input-10-d49f76dd3d33>:22(find_duplicate_movies)
        1    0.000    0.000    0.002    0.002 <ipython-input-10-d49f76dd3d33>:12(read_movies)
        1    0.001    0.001    0.001    0.001 <ipython-input-10-d49f76dd3d33>:24(<listcomp>)
        1    0.000    0.000    0.001    0.001 {method 'read' of '_io.TextIOWrapper' objects}
     5043    0.001    0.000    0.001    0.000 {method 'lower

## Observation 

Futher Reduction in runtime() from 0.34 sec to 0.15 sec, Thats how we use cprofile to identify such bottlenecks, With just 2 small changes we have exponentially reduced the runtime from 5 sec to 0.15 sec 

Now we know that cprofile has got some overhead, Therefore the actual execution time will be surely lesser than 0.15 sec

# Memory Profiling

For memory profiling we use the module memory profiler, this one is rarely used. We'll look at the execution and print memory stats for the same program that was used above, This is simple and straight fwd 


In [11]:
start = time.time()

### IMPORTING MEMORY PROFILE 
from memory_profiler import profile


def read_movies(src):
    with open(src) as datafile:
        return datafile.read().splitlines()

    
@profile       # use decorator "profile" for the Target function 
def find_duplicate_movies(src= 'movie_metadata.txt'):
    movies = read_movies(src)
    movies = [movie.lower() for movie in movies]  
    duplicates = []
    while movies:
        this_movie = movies.pop()
        if this_movie in movies:       # Changed Here 
            duplicates.append(this_movie)
    return duplicates

duplicates = find_duplicate_movies()
print ("Number of Duplicates : ",len(duplicates))

end = time.time()
print("exec_time = ",end-start," sec.")


ERROR: Could not find file <ipython-input-11-39b04b1b9f69>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.
Number of Duplicates :  127
exec_time =  0.19238018989562988  sec.


Looks like Memory profilers doesn't work in ipython env, We'll need the terminal interface to look at it, The report looks like the one included in the markdown below

```
Filename: .\tutorial.py

Line #    Mem usage    Increment   Line Contents
================================================
    21     28.4 MiB     28.4 MiB   @profile  #>>  Decorator for the target function  >> Uncomment this for memory profile
    22                             def find_duplicate_movies(src= 'movie_metadata.txt'):
    23     28.8 MiB      0.4 MiB       movies = read_movies(src)
    24     29.1 MiB      0.1 MiB       movies = [movie.lower() for movie in movies]
    25     29.1 MiB      0.0 MiB       duplicates = []
    26     29.1 MiB      0.0 MiB       while movies:
    27     29.1 MiB      0.0 MiB           this_movie = movies.pop()
    28     29.1 MiB      0.0 MiB           if this_movie in movies:
    29     29.1 MiB      0.0 MiB               duplicates.append(this_movie)
    30     29.1 MiB      0.0 MiB       return duplicates

```